# Offline monitoring

This Jupyter Notebook helps to visualize the L60 data monitoring plots.

In [11]:
%matplotlib widget
import os, sys, json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import dates
import pickle as pkl
import ipywidgets as widget
import ipydatetime
import datetime
import numpy as np
from datetime import datetime, timezone

In [12]:
def get_day_hour(date):
    start = date.split('_')[0][:-1]
    stop = date.split('_')[1][:-1]
    start_day = datetime.strptime(start,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    stop_day = datetime.strptime(stop,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
    
    return start_day+" -> "+stop_day

## Load data
In the first step, you automatically select data that were generated with the latest version of ```settings/config.json```. (add a manual definition of pkl_name? what if someone wants to analyze sth different from what was generated last time?)

At the moment this block of code just reads info about the last generated data to retrieve exp number, period, run and datatype.
The remaining information (time window of the analysis, parameters list, string number and barrel position) are automatically read from the name of files in the output folder: in this way every plot that has been generated and saved in the output folder can be accessed.

Plot selection can be easily extended to allow selection of exp, period, run and type of data acquisition (phys, cal, other) simply from reading the name of the files in the output folder.

In [14]:
with open("config.json") as f: 
    j_config = json.load(f)

exp         = j_config['run_info']['exp']
period      = j_config['period']
run         = j_config['run']
datatype    = j_config['datatype']

pkl_name = exp + '-' + period + '-' + run + '-' + datatype 
print("The following analysis will be inspected:", pkl_name)

The following analysis will be inspected: l60-p01-r006-phy


Let's load the pickle files having the ```pkl_name``` in it. Germanium detectors and SiPMs will be separately analyzed (considering different parameters are plotted for each one of them and different "string" layouts are adopted).

In [15]:
pkl_files = os.listdir('out/pkl-files/par-vs-time/')

geds_list = [file for file in pkl_files if 'string' in file]      #geds: grouped by string
spms_list = [file for file in pkl_files if 'string' not in file]  #spms: everything not grouped by string

if geds_list != []:
    geds_map = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in geds_list]))
    geds_par = list(dict.fromkeys([file.split('-')[-2] for file in geds_list]))
    geds_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in geds_list])))
    print("geds map:", geds_map)
    print("geds par:", geds_par)
else: print("-> NO data for geds was found")

geds_date_formatted = [get_day_hour(date) for date in geds_date]
geds_time_option = [(key, value) for key,value in zip(geds_date_formatted, geds_date)]


if spms_list != []:
    spms_map = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list]))
    spms_par = list(dict.fromkeys([file.split('-')[-2] for file in spms_list]))
    spms_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in spms_list])))
    print("spms map:", spms_map)
    print("spms par:", spms_par)
else:     print("-> NO data for spms was found")   

spms_date_formatted = [get_day_hour(date) for date in spms_date]
spms_time_option = [(key, value) for key,value in zip(spms_date_formatted, spms_date)]

geds map: ['string1', 'string2', 'string3', 'string4', 'string5', 'string6', 'string7']
geds par: ['baseline']
spms map: ['top_OB', 'bot_OB', 'top_IB', 'bot_IB']
spms par: ['baseline', 'daqenergy']


## Interactive widgets
Define here the interactive widgets you want to include in the final plots. Data+time selection widget still needs to be implemented.

In [16]:
# tab for selecting what to plot: par vs time, heatmap
plot_type_buttons = widget.ToggleButtons(
    options= ['par-vs-time', 'heatmap'],
    description='Plot type:',
    disabled=False,
    button_style='', 
    tooltips=[]
)

# select widget for selecting the date (for geds) 
geds_time_select = widget.Select(
    options=geds_time_option,
    value=geds_date[0],
    # rows=10,
    description='Time:',
    disabled=False,
    layout={'width': 'max-content'}
)

# select widget for selecting the date (for spms)
spms_time_select = widget.Select(
    options=spms_time_option,
    value=spms_date[0],
    # rows=10,
    description='Time:',
    disabled=False,
    layout={'width': 'max-content'}
)


# tab for selecting parameters to plot (for geds)
par_geds_buttons = widget.ToggleButtons(
    options= geds_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# tab for selecting parameters to plot (for spms)
par_spms_buttons = widget.ToggleButtons(
    options= spms_par,#par_spms,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# tab for selecting the string number (for geds)
geds_map_buttons = widget.ToggleButtons(
    options= geds_map,
    description='String #:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# tab for selecting the string number (for spms)
spms_map_buttons = widget.ToggleButtons(
    options= spms_map,
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

## Germanium detectors - plots

In [17]:
# geds
def plot_geds(plot_type, time_select, parameter, string):
    ax = pkl.load(open(f'pkl-files/{plot_type}/{pkl_name}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    ax.plot(figsize=(50,50))
    plt.tight_layout()
    plt.show()
    
if geds_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()
out_geds = widget.interactive_output(plot_geds, {'plot_type'  :plot_type_buttons,
                                                 'time_select':geds_time_select,
                                                 'parameter'  :par_geds_buttons, 
                                                 'string'     :geds_map_buttons})

widget.VBox([widget.HBox([geds_time_select,widget.VBox([plot_type_buttons, par_geds_buttons, geds_map_buttons])]),out_geds])

## SiPM detectors - plots

In [8]:
# spms
def plot_spms(plot_type, time_select, parameter, string):
    ax = pkl.load(open(f'pkl-files/{plot_type}/{pkl_name}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    if parameter=='gain': ax[0].plot(figsize=(50,50))
    else: ax.plot(figsize=(50,50))
    plt.tight_layout()
    plt.show()
    
if spms_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()   
out_spms = widget.interactive_output(plot_spms, {'plot_type'  :plot_type_buttons,
                                                 'time_select':spms_time_select,
                                                 'parameter'  :par_spms_buttons,
                                                 'string'     :spms_map_buttons})

widget.VBox([spms_time_select, plot_type_buttons, par_spms_buttons, spms_map_buttons, out_spms]) 
widget.VBox([widget.HBox([spms_time_select,widget.VBox([plot_type_buttons, par_spms_buttons, spms_map_buttons])]),out_spms])